<a href="https://colab.research.google.com/github/sileod/tasknet/blob/main/Easy_multi_task_ModernBERT_fine_tuning_with_tasknet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.system('pip install triton accelerate tasknet tasksource')
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import tasknet as tn
from datasets import load_dataset
import datasets
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*seems not to be NE tag.*") #seqeval issue

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
class args:
    model_name = "sileod/deberta-v3-small-tasksource-nli"
    model_name = "tasksource/ModernBERT-base-nli"
    # remaining arguments are from https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/trainer#transformers.TrainingArguments
    learning_rate = 3e-5
    #max_steps = 500
    num_train_epochs =2



mrpc = tn.Classification(
    dataset=load_dataset("glue", "mrpc"), s1="sentence1", s2="sentence2", y="label"
)

wnli = tn.AutoTask('glue/wnli',main_split="validation")


conll_pos = tn.TokenClassification(
    dataset="conll2003", # A dataset string/tuple can also be passed and loaded internally with datasets.load_dataset
    name="conll_pos", tokens="tokens", y="pos_tags"
)


reco = tn.MultipleChoice(
    tn.utils.train_validation_test_split(
        tn.load_dataset("sileod/movie_recommendation")["test"]
    ),
    s1="question",
    y="label",
    choices=["answer_0", "answer_1", "answer_2", "answer_3"],
)

stsb = tn.AutoTask("glue/stsb") # https://github.com/sileod/tasksource/blob/main/tasks.md

tasks = [wnli,mrpc]
models = tn.Model(tasks, args) # list of models; by default, shared encoder, task-specific CLS token task-specific head
trainer = tn.Trainer(models, tasks, args) # tasks are uniformly sampled by default
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/variables/functions.py:725: UserWarning: Graph break due to unsupported builtin None.Random.random. This function is either a Python builtin (e.g. _warnings.warn) or a third-party C/C++ Python extension (perhaps created with pybind). If it is a Python builtin, please file an issue on GitHub so the PyTorch team can add support for it and see the next case for a workaround. If it is a third-party C/C++ Python extension, please either wrap it into a PyTorch-understood custom operator (see https://pytorch.org/tutorials/advanced/custom_ops_landing_page.html for more details) or, if it is traceable, use torch.compiler.allow_in_graph.
  torch._dynamo.utils.warn_once(msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Name,Size,Index
1,0.371400,0.839300,0.514286,0.499579,glue/wnli,35,0
1,0.371400,0.351788,0.882353,0.864527,glue/mrpc,408,1
2,0.102600,1.308422,0.514286,0.512695,glue/wnli,35,0
2,0.102600,0.701844,0.892157,0.870139,glue/mrpc,408,1


TrainOutput(global_step=1078, training_loss=0.23698728911730707, metrics={'train_runtime': 237.4655, 'train_samples_per_second': 36.241, 'train_steps_per_second': 4.54, 'total_flos': 2948257146052608.0, 'train_loss': 0.23698728911730707, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.7018182873725891,
 'eval_accuracy': 0.8921568627450981,
 'eval_f1': 0.8701388888888889,
 'eval_name': 'glue/mrpc',
 'eval_size': 408,
 'eval_index': 1,
 'eval_runtime': 3.2109,
 'eval_samples_per_second': 127.069,
 'eval_steps_per_second': 15.884,
 'epoch': 2.0,
 'Accuracy': nan}